# Explore here

It's recommended to use this notebook for exploration purposes.

For example: 

1. You could import the CSV generated by python into your notebook and explore it.
2. You could connect to your database using `pandas.read_sql` from this notebook and explore it.

In [65]:
! pip install pandas
! pip install regex
! pip install matplotlib
! pip install nltk



[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip


In [66]:
! pip install sklearn


[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip


In [67]:

import pandas as pd 
import regex as reg
import re
import matplotlib.pyplot as plt
import unicodedata
import nltk
import string
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

from sklearn import model_selection, svm
from sklearn.metrics import classification_report, accuracy_score
from nltk.corpus import stopwords



In [68]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv')

In [69]:
df_raw.info

<bound method DataFrame.info of                                                     url  is_spam
0     https://briefingday.us8.list-manage.com/unsubs...     True
1                                https://www.hvper.com/     True
2                    https://briefingday.com/m/v4n3i4f3     True
3      https://briefingday.com/n/20200618/m#commentform    False
4                           https://briefingday.com/fan     True
...                                                 ...      ...
2994  https://www.smartcitiesworld.net/news/news/dee...    False
2995                      https://www.youtube.com/watch     True
2996  https://techcrunch.com/2019/07/04/an-optimisti...    False
2997  https://www.technologyreview.com/2019/12/20/13...    False
2998       https://www.bbc.com/news/technology-51018758    False

[2999 rows x 2 columns]>

In [70]:
#Vamos a pasar a 0 y 1 la variable objetivo

df_raw['is_spam'] = df_raw['is_spam'].apply(lambda x: 1 if x == True else 0)

In [71]:
df_raw.info

<bound method DataFrame.info of                                                     url  is_spam
0     https://briefingday.us8.list-manage.com/unsubs...        1
1                                https://www.hvper.com/        1
2                    https://briefingday.com/m/v4n3i4f3        1
3      https://briefingday.com/n/20200618/m#commentform        0
4                           https://briefingday.com/fan        1
...                                                 ...      ...
2994  https://www.smartcitiesworld.net/news/news/dee...        0
2995                      https://www.youtube.com/watch        1
2996  https://techcrunch.com/2019/07/04/an-optimisti...        0
2997  https://www.technologyreview.com/2019/12/20/13...        0
2998       https://www.bbc.com/news/technology-51018758        0

[2999 rows x 2 columns]>

In [72]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      2999 non-null   object
 1   is_spam  2999 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


In [73]:
df_raw['url'].sample(20)

1806                           https://justiceinjune.org/
1483             http://us2.forward-to-friend.com/forward
2908    https://www.nytimes.com/2020/06/30/world/asia/...
681                          https://www.morningbrew.com/
604                         https://shop.morningbrew.com/
44      https://ourworldindata.org/grapher/daily-covid...
1988    https://podcasts.apple.com/us/podcast/ask-ronn...
2149    https://www.reuters.com/article/us-health-coro...
2860    https://numlock.substack.com/p/2018-sunday-spe...
1728     https://thehustle.co/06292020-butchers-COVID-19/
1437    https://www.wsj.com/articles/pepsico-unit-to-r...
1974    https://www.vox.com/policy-and-politics/2020/6...
1965    https://www.masslive.com/sports/2020/06/colleg...
434     https://www.reuters.com/article/us-usa-stocks/...
1577      https://www.physics.harvard.edu/undergrad/humor
2805    https://briefingday.us8.list-manage.com/unsubs...
682      https://bigspaceship.us9.list-manage.com/profile
2196    https:

In [74]:
df_raw['is_spam'].value_counts()

0    2303
1     696
Name: is_spam, dtype: int64

In [75]:
# Vamos a ver como esta nuestra variable objetivo en este caso es is_spam
df_raw['is_spam'].value_counts()
print("Hay spam: ",len(df_raw.loc[df_raw.is_spam==1]))
print("Que no son spam : ",len(df_raw.loc[df_raw.is_spam==0]))
print(df_raw.shape)


Hay spam:  696
Que no son spam :  2303
(2999, 2)


In [76]:
# Eliminar los duplicados

df_raw = df_raw.drop_duplicates()
df_raw = df_raw.reset_index(inplace = False)[['url','is_spam']]
df_raw.shape

(2369, 2)

Empezamos el proceso de Limipieza de los datos

In [77]:
# colocar todos los textos en minusculas
df_raw['url'] = df_raw['url'].str.lower()

In [79]:
df_aux = df_raw.copy()

In [80]:

# Proceso de limpieza de datos

limpieza = []

for p in range(len(df_aux.url)):
    desc = df_aux['url'][p]
    
    #sacar la puntuacion
    desc = re.sub('[^a-zA-Z]', ' ', desc)
    
    #borrar etiquetas especiales
    desc=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",desc)
    
    #borrar digitos y caracteres especiales
    desc=re.sub("(\\d|\\W)+"," ",desc)
    
    limpieza.append(desc)

#asiganmos los datos limipios en limpieza
df_aux['url'] = limpieza
        
df_aux.head()

,url,is_spam
0,https briefingday us list manage com unsubscribe,1
1,https www hvper com,1
2,https briefingday com m v n i f,1
3,https briefingday com n m commentform,0
4,https briefingday com fan,1


In [81]:
df_aux['url'].str.split(expand=True).stack().value_counts()[:60]

https          2336
com            2065
www            1512
the             354
html            296
news            274
a               252
us              248
to              218
of              173
coronavirus     172
e               150
org             146
c               136
article         131
b               124
in              115
and             113
morningbrew     105
story           105
nytimes         101
on              101
daily            99
d                98
stories          94
utm              90
for              90
youtube          89
v                89
trump            88
numlock          87
watch            86
f                78
is               77
new              76
p                69
world            68
substack         68
reuters          65
covid            63
s                62
briefingday      61
index            61
vox              59
en               59
articles         58
cnn              58
iduskbn          58
politics         56
co               56


In [82]:
# Creamos una lista de stopwords
stop_words = ['http','www','com','you','your','for','not','have','is','in','im','from','to','https','e','c','v','b','f','p']

In [83]:

for i in df_aux['url'].str.split(expand=True).stack().value_counts().index:
    if len(i)<3 :
        stop_words.append(i)


In [84]:
stop_words=list(set(stop_words))
len(stop_words)

268

In [85]:

def remove_stopwords(message):
  if message is not None:
    words = message.strip().split()
    words_filtered = []
    for word in words:
      if word not in stop_words:
        words_filtered.append(word) 
    result = " ".join(words_filtered)         
  else:
    result = None

  return result 


In [86]:
df_aux['url']=df_aux['url'].apply(remove_stopwords)

In [87]:
df_aux.sample(10)

,url,is_spam
2278,gimletmedia shows reply all,0
1377,lelo pride month,0
805,nytimes opinion sunday why trump cant disengag...,0
1249,austinkleon list manage unsubscribe,1
277,inverse mind body the first stay home dad,0
1600,wired article fbi forensic photos,0
1716,cosmopolitan lifestyle jenna marbles quitting ...,0
2125,level medium breonna taylor meme,0
572,morningbrew daily,0
120,theatlantic technology archive how youll shop ...,0


In [88]:
df_aux['url'].str.split(expand=True).stack().value_counts()[:60]

the            354
html           296
news           274
coronavirus    172
org            146
article        131
and            113
morningbrew    105
story          105
nytimes        101
daily           99
stories         94
utm             90
youtube         89
trump           88
numlock         87
watch           86
new             76
substack        68
world           68
reuters         65
covid           63
briefingday     61
index           61
vox             59
cnn             58
articles        58
iduskbn         58
politics        56
cnbc            54
sunday          51
business        49
court           48
apnews          47
email           46
facebook        46
health          45
bbc             41
supreme         41
blog            40
are             40
black           39
medium          39
npr             38
police          38
digg            37
campaign        37
with            37
why             36
theverge        35
apple           35
cases           35
theguardian 

In [89]:
# Ahora que hemos limpiado nuestros datos
# procedemos a hacer una copia con el dataset a trabajar en el final
df = df_aux.copy()

In [90]:
X = df['url']
y = df['is_spam']
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,random_state=121)

In [91]:
# Vectorizador
vec = CountVectorizer(stop_words='english')
X_train = vec.fit_transform(X_train).toarray()
X_test = vec.transform(X_test).toarray()

In [92]:
print(X_train)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [93]:
X_train.shape

(1776, 4658)

In [94]:
X_test.shape

(593, 4658)

In [95]:
print(vec.get_feature_names_out())

['aab' 'aaron' 'abacus' ... 'zulalimtm' 'zwift' 'zwn']


In [96]:
nb = MultinomialNB()

In [115]:
nb.fit(X_train, y_train)

MultinomialNB()

In [116]:
predictions = nb.predict(X_train)
print(classification_report(y_train, predictions))

              precision    recall  f1-score   support

       False       0.92      0.70      0.79      1839
        True       0.44      0.79      0.57       560

    accuracy                           0.72      2399
   macro avg       0.68      0.74      0.68      2399
weighted avg       0.80      0.72      0.74      2399



In [117]:
predictions = nb.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

       False       0.92      0.72      0.81       464
        True       0.45      0.79      0.58       136

    accuracy                           0.74       600
   macro avg       0.69      0.75      0.69       600
weighted avg       0.81      0.74      0.76       600



In [118]:
#Creando la Matriz

message_vectorizer = CountVectorizer().fit_transform(df['url'])

# Haciendo el split de los datos

X_train, X_test, y_train, y_test = train_test_split(message_vectorizer, df['is_spam'], test_size = 0.2, random_state = 121, shuffle = True)

In [119]:
X_train.shape

(2399, 6895)

In [120]:
cl = svm.SVC(C=1.0, kernel='linear', degree=4, gamma='auto')

In [122]:
cl.fit(X_train, y_train)

pred = cl.predict(X_train)
print(classification_report(y_train, pred))

              precision    recall  f1-score   support

       False       0.99      1.00      1.00      1842
        True       1.00      0.98      0.99       557

    accuracy                           0.99      2399
   macro avg       1.00      0.99      0.99      2399
weighted avg       1.00      0.99      0.99      2399



In [123]:
pred = cl.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

       False       0.97      0.98      0.97       461
        True       0.93      0.90      0.91       139

    accuracy                           0.96       600
   macro avg       0.95      0.94      0.94       600
weighted avg       0.96      0.96      0.96       600



Este último es el mejor resultado obtenido, por lo que considero guardar

In [105]:
pickle.dump(cl, open('../models/texto_NLP.pkl', 'wb'))

In [106]:
# Si queremos cargar el archivo guardado en la carpeta models

load_model = pickle.load(open('../models/texto_NLP.pkl', 'rb'))

In [107]:
load_model

SVC(degree=4, gamma='auto', kernel='linear')

Ahora probe solo cambiando a las columnas, y el resultado al final no es el mejor

In [108]:
df = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv')

In [109]:
df['len_url'] = df['url'].apply(lambda x : len(x))
df['contains_subscribe'] = df['url'].apply(lambda x : 1 if "subscribe" in x else 0)
df['contains_hash'] = df['url'].apply(lambda x : 1 if "#" in x else 0)
df['num_digits'] = df['url'].apply(lambda x : len("".join(_ for _ in x if _.isdigit())) )
df['non_https'] = df['url'].apply(lambda x : 1 if "https" in x else 0)
df['num_words'] = df['url'].apply(lambda x : len(x.split("/")))

target = 'is_spam'
features = [f for f in df.columns if f not in ["url", target]]
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=0)

In [110]:
X_train.head()

,len_url,contains_subscribe,contains_hash,num_digits,non_https,num_words
1521,74,0,0,0,1,6
1473,34,0,0,0,1,5
731,114,0,0,12,1,8
271,51,0,0,20,1,4
1509,43,0,0,0,1,5


In [111]:
df['contains_hash'].value_counts()

0    2943
1      56
Name: contains_hash, dtype: int64

In [112]:
cl = svm.SVC(C=1.0, kernel='linear', degree=4, gamma='auto')

In [113]:
cl.fit(X_train, y_train)
pred = cl.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

       False       0.91      0.90      0.91       464
        True       0.67      0.70      0.69       136

    accuracy                           0.85       600
   macro avg       0.79      0.80      0.80       600
weighted avg       0.86      0.85      0.86       600



In [ ]:
Esta prueba no es la mejor, pero el tema que tenemos mas columnas con las palabras separadas que cantidad de observaciones
lo que se hace dificil predecir algo en este caso